In [275]:
# Import dependencies
import requests
import os
import pprint
from datetime import datetime,timedelta
import json
import pandas as pd
from dateutil.parser import parse
import itertools
from itertools import permutations
import tqdm 
from tqdm import tqdm
import csv


In [352]:
# Set params for query
pax = input(' # of adults will be traveling?')
children = input(' # Children traveling with you?')
p0 = input(' Starting place') 
startdate = input('Trip start date (mm/dd/yyyy)')
destinations = input('List the cities do you want to travel to? And how many days per city? ex(London,4,Madrid,3)')
#destinations.replace(" ", "")
destinations1 = list(destinations.split(','))




 # of adults will be traveling?1
 # Children traveling with you?0
 Starting placemiami
Trip start date (mm/dd/yyyy)12/10/2019
List the cities do you want to travel to? And how many days per city? ex(London,4,Madrid,3)madrid,3,paris,5,tel aviv,6


In [353]:
p1=(str(destinations1[0]),int(destinations1[1]))
p2 = (str(destinations1[2]), int(destinations1[3]))
p3 = (str(destinations1[4]), int(destinations1[5]))


def changeorder(res,p1,p2,p3,startdate,p0,children,pax):
    

    start_date = parse(startdate)
    days_p1=p1[1]
    days_p2=p2[1]
    days_p3=p3[1]
    start_p1 = start_date + timedelta(days=days_p1)
    start_p2 = start_p1 + timedelta(days=days_p2)
    start_p3 = start_p2 + timedelta(days=days_p3)
    url = "https://apidojo-hipmunk-v1.p.rapidapi.com/flights/create-session"

    querystring = {"from1":p1[0],"from2":p2[0],"from3":p3[0],"to1":p2[0],"to2":p3[0],"to3":p0,"date1":start_p1,"date2":start_p2,"date3":start_p3,"infants_lap":"0","children":children,"seniors":"0","country":"US","from0":p0,"to0":p1[0],"date0":startdate,"pax":pax,"cabin":"Coach"}
    headers = {
        'x-rapidapi-host': "apidojo-hipmunk-v1.p.rapidapi.com",
        'x-rapidapi-key': "c36e6ce6fcmsh7fe55ace5e9f40fp10fdc1jsn690a20bbc0ef"
        }

    response = requests.request("GET", url, headers=headers, params=querystring).json()

    #does stuff with response
    idnums=[]
    agonies=[]
    prices=[]
    routingid=[]
    

    for key,value in response['itins'].items():
        idnum=key
        agony=value['agony']
        price=value['price']
        routing_id=value['routing_idens']
        idnums.append(idnum)
        agonies.append(agony)
        prices.append(price)
        routingid.append(routing_id)
    df=pd.DataFrame()
    df['price']=prices
    df['agonies']=agonies
    df['id']=idnums
    df['routing']=routingid
    
    
    
    


    # Print DF with values
    df = df.sort_values(['price','agonies'], ascending=[True,True])
    df = df[:1]

    res.append([(p1,p2,p3),df])



    #p1,p2,p3 = p2,p3,p1
    return res
    
    

In [354]:
alist = []
listA = [p1, p2, p3]
perm = itertools.permutations(listA) 
  
for i in list(perm): 
    alist.append(i)
alist
    
    
    

[(('madrid', 3), ('paris', 5), ('tel aviv', 6)),
 (('madrid', 3), ('tel aviv', 6), ('paris', 5)),
 (('paris', 5), ('madrid', 3), ('tel aviv', 6)),
 (('paris', 5), ('tel aviv', 6), ('madrid', 3)),
 (('tel aviv', 6), ('madrid', 3), ('paris', 5)),
 (('tel aviv', 6), ('paris', 5), ('madrid', 3))]

In [355]:
print('Sorry, this may take a while...')
res=[]
for i in tqdm(alist):
    p1,p2,p3=i[0],i[1],i[2]
    res=changeorder(res,p1,p2,p3,startdate,p0,children,pax)
    
res=[]
for i in tqdm(alist):
    p1,p2,p3=i[0],i[1],i[2]
    res=changeorder(res,p1,p2,p3,startdate,p0,children,pax)    
    
print('Search 100% completed')   
res

  0%|          | 0/6 [00:00<?, ?it/s]

Sorry, this may take a while...


100%|██████████| 6/6 [01:05<00:00, 10.89s/it]

Search 100% completed


[[(('madrid', 3), ('paris', 5), ('tel aviv', 6)),
        price      agonies                                id  \
  451  1360.0  3366.556982  1e57642a6365eb4527ab9e439d96ca5d   
  
                                                 routing  
  451  [6b5eafacd8e3c9973a0693a7b163a7ea, 4da163beb4d...  ],
 [(('madrid', 3), ('tel aviv', 6), ('paris', 5)),
         price      agonies                                id  \
  4220  1169.0  2691.489159  4dbb0296a806ba1d19aae4aaa2d66ff4   
  
                                                  routing  
  4220  [6b5eafacd8e3c9973a0693a7b163a7ea, 41f78d59aa8...  ],
 [(('paris', 5), ('madrid', 3), ('tel aviv', 6)),
       price      agonies                                id  \
  63  1352.0  3394.656868  d67d88cdfdce89c8ebbec12689e6202a   
  
                                                routing  
  63  [a3e1e922f052f49906be7194859f7da3, 03c40d6c062...  ],
 [(('paris', 5), ('tel aviv', 6), ('madrid', 3)),
         price      agonies                    

In [356]:
df1=pd.DataFrame(res[0][1])
df2=pd.DataFrame(res[1][1])
df3=pd.DataFrame(res[2][1])
df4=pd.DataFrame(res[3][1])
df5=pd.DataFrame(res[4][1])
df6=pd.DataFrame(res[5][1])


In [357]:
#Joining all results into a single DF:

#Option 1
first_choice = df1['routing'].astype(str)
df1new = first_choice.str.split(", ",expand=True)
firstchoice_merged = df1.merge(df1new, how='outer', left_index=True, right_index=True)
firstchoice_merged = firstchoice_merged.drop(columns=['routing'])

#Option 2
second_choice = df2['routing'].astype(str)
df2new = second_choice.str.split(", ",expand=True)
secondchoice_merged = df2.merge(df2new, how='outer', left_index=True, right_index=True)
secondchoice_merged = secondchoice_merged.drop(columns=['routing'])

#Option 3
third_choice = df3['routing'].astype(str)
df3new = third_choice.str.split(", ",expand=True)
thirdchoice_merged = df3.merge(df3new, how='outer', left_index=True, right_index=True)
thirdchoice_merged = thirdchoice_merged.drop(columns=['routing'])

#Option 4
fourth_choice = df4['routing'].astype(str)
df4new = fourth_choice.str.split(", ",expand=True)
fourthchoice_merged = df4.merge(df4new, how='outer', left_index=True, right_index=True)
fourthchoice_merged = fourthchoice_merged.drop(columns=['routing'])

#Option 5
fifth_choice = df5['routing'].astype(str)
df5new = fifth_choice.str.split(", ",expand=True)
fifthchoice_merged = df5.merge(df5new, how='outer', left_index=True, right_index=True)
fifthchoice_merged = fifthchoice_merged.drop(columns=['routing'])

#Option 6
sixth_choice = df6['routing'].astype(str)
df6new = sixth_choice.str.split(", ",expand=True)
sixthchoice_merged = df6.merge(df6new, how='outer', left_index=True, right_index=True)
sixthchoice_merged = sixthchoice_merged.drop(columns=['routing'])

bigdata = [firstchoice_merged,secondchoice_merged,thirdchoice_merged,fourthchoice_merged,fifthchoice_merged,sixthchoice_merged]
result = pd.concat(bigdata)
#result_df = result.sort_values(['price','agonies'], ascending=[True,True])

In [358]:
resultuse_df = result.drop(columns=[0,1,2,3])
resultuse_df


,price,agonies,id
451,1360.0,3366.556982,1e57642a6365eb4527ab9e439d96ca5d
4220,1169.0,2691.489159,4dbb0296a806ba1d19aae4aaa2d66ff4
63,1352.0,3394.656868,d67d88cdfdce89c8ebbec12689e6202a
1468,1069.0,2928.916395,16001a6dca76342afb2d8893d99af575
251,1191.0,2905.943328,c187bac451a7a344b35132e22f4ad6ac
1901,1119.0,3007.443690,620d030709a294a71a9936366f58672b


In [359]:
xlist1=[]
ylist1=[]
xlist2=[]
ylist2=[]
xlist3=[]
ylist3=[]

for x,y,z in alist:
    d1,d2 = x
    xlist1.append(d1)
    ylist1.append(d2)
    d3,d4 = y
    xlist2.append(d3)
    ylist2.append(d4)
    d5,d6 = z
    xlist3.append(d5)
    ylist3.append(d6)
print(xlist1)
print(ylist1)



['madrid', 'madrid', 'paris', 'paris', 'tel aviv', 'tel aviv']
[3, 3, 5, 5, 6, 6]


In [360]:
alist

[(('madrid', 3), ('paris', 5), ('tel aviv', 6)),
 (('madrid', 3), ('tel aviv', 6), ('paris', 5)),
 (('paris', 5), ('madrid', 3), ('tel aviv', 6)),
 (('paris', 5), ('tel aviv', 6), ('madrid', 3)),
 (('tel aviv', 6), ('madrid', 3), ('paris', 5)),
 (('tel aviv', 6), ('paris', 5), ('madrid', 3))]

In [361]:
alist_df = pd.DataFrame(alist)
alist_df
alist_df

,0,1,2
0,"(madrid, 3)","(paris, 5)","(tel aviv, 6)"
1,"(madrid, 3)","(tel aviv, 6)","(paris, 5)"
2,"(paris, 5)","(madrid, 3)","(tel aviv, 6)"
3,"(paris, 5)","(tel aviv, 6)","(madrid, 3)"
4,"(tel aviv, 6)","(madrid, 3)","(paris, 5)"
5,"(tel aviv, 6)","(paris, 5)","(madrid, 3)"


In [362]:
start = parse(startdate)
#start = start_date.strftime("%B %d, %Y")

#Translating the lenght of stay into a date for first stop departure
g=[]
date1=[]
for days1 in ylist1:
    g = start + timedelta(days=days1)
    #g1 = g.strftime("%B %d, %Y")
    date1.append(g)

#Translating the lenght of stay into a date for second stop departure
c=[]
date2=[]
c1=[]
c2=[]
i=[]

for days2 in date1:
    c1.append(days2)

for i in range(6):    
    xi = (c1[i])
    xh = (ylist2[i])
    c2 = xi + timedelta(days=xh)
    date2.append(c2) 
    
#Translating the lenght of stay into a date for third stop departure    
    
c3=[]
date3=[]
c3=[]
c4=[]
i=[]

for days3 in date2:
    c3.append(days3)

for i in range(6):    
    xe = (c3[i])
    xr = (ylist3[i])
    c4 = xe + timedelta(days=xr)
    date3.append(c4)

                               
resultuse_df['Starting_Point'] = p0
resultuse_df['Starting_Date'] = start
resultuse_df['First_Stop'] = xlist1
resultuse_df['Departure_1'] = date1
resultuse_df['Second_Stop'] = xlist2
resultuse_df['Departure_2'] = date2
resultuse_df['Third_Stop'] = xlist3
resultuse_df['Departure_3'] = date3
resultuse_df['End_Point'] = p0
#resultuse_df = resultuse_df.drop[['Starting_Point','Starting_Date']]
df_full = resultuse_df[['price','agonies','id','Starting_Point','Starting_Date','First_Stop','Departure_1','Second_Stop','Departure_2','Third_Stop','Departure_3','End_Point']]

In [363]:
df_fullnew = df_full.rename(columns={"id":'Itinerary_#'})
df_fullnew=df_fullnew.sort_values(['price','agonies'], ascending=[True,True])

In [364]:
df_fullnew

,price,agonies,Itinerary_#,Starting_Point,Starting_Date,First_Stop,Departure_1,Second_Stop,Departure_2,Third_Stop,Departure_3,End_Point
1468,1069.0,2928.916395,16001a6dca76342afb2d8893d99af575,miami,2019-12-10,paris,2019-12-15,tel aviv,2019-12-21,madrid,2019-12-24,miami
1901,1119.0,3007.443690,620d030709a294a71a9936366f58672b,miami,2019-12-10,tel aviv,2019-12-16,paris,2019-12-21,madrid,2019-12-24,miami
4220,1169.0,2691.489159,4dbb0296a806ba1d19aae4aaa2d66ff4,miami,2019-12-10,madrid,2019-12-13,tel aviv,2019-12-19,paris,2019-12-24,miami
251,1191.0,2905.943328,c187bac451a7a344b35132e22f4ad6ac,miami,2019-12-10,tel aviv,2019-12-16,madrid,2019-12-19,paris,2019-12-24,miami
63,1352.0,3394.656868,d67d88cdfdce89c8ebbec12689e6202a,miami,2019-12-10,paris,2019-12-15,madrid,2019-12-18,tel aviv,2019-12-24,miami
451,1360.0,3366.556982,1e57642a6365eb4527ab9e439d96ca5d,miami,2019-12-10,madrid,2019-12-13,paris,2019-12-18,tel aviv,2019-12-24,miami


In [365]:
destination_df = df_fullnew.loc[:,['Starting_Point','First_Stop','Second_Stop','Third_Stop','End_Point']]
destination_df

,Starting_Point,First_Stop,Second_Stop,Third_Stop,End_Point
1468,miami,paris,tel aviv,madrid,miami
1901,miami,tel aviv,paris,madrid,miami
4220,miami,madrid,tel aviv,paris,miami
251,miami,tel aviv,madrid,paris,miami
63,miami,paris,madrid,tel aviv,miami
451,miami,madrid,paris,tel aviv,miami


In [366]:
# Create an empty list 
destinations_list =[] 
  
# Iterate over each row 
for index, rows in destination_df.iterrows(): 
    # Create list for the current row 
    my_list =[rows.Starting_Point, rows.First_Stop, rows.Second_Stop, rows.Third_Stop, rows.End_Point] 
      
    # append the list to the final list 
    destinations_list.append(my_list) 
  
# Print the list 
print(destinations_list) 

[['miami', 'paris', 'tel aviv', 'madrid', 'miami'], ['miami', 'tel aviv', 'paris', 'madrid', 'miami'], ['miami', 'madrid', 'tel aviv', 'paris', 'miami'], ['miami', 'tel aviv', 'madrid', 'paris', 'miami'], ['miami', 'paris', 'madrid', 'tel aviv', 'miami'], ['miami', 'madrid', 'paris', 'tel aviv', 'miami']]


In [378]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="specify_your_app_name_here")

from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode)
destination_df['location'] = destination_df['Starting_Point'].apply(geocode)
destination_df['point'] = destination_df['location'].apply(lambda loc: tuple(loc.point))
destination_df['location2'] = destination_df['First_Stop'].apply(geocode)
destination_df['point2'] = destination_df['location2'].apply(lambda loc: tuple(loc.point) if loc else None)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('miami',), **{}).
Traceback (most recent call last):
  File "/Users/cla/opt/anaconda3/lib/python3.7/urllib/request.py", line 1317, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/Users/cla/opt/anaconda3/lib/python3.7/http/client.py", line 1244, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/Users/cla/opt/anaconda3/lib/python3.7/http/client.py", line 1290, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/Users/cla/opt/anaconda3/lib/python3.7/http/client.py", line 1239, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/Users/cla/opt/anaconda3/lib/python3.7/http/client.py", line 1026, in _send_output
    self.send(msg)
  File "/Users/cla/opt/anaconda3/lib/python3.7/http/client.py", line 966, in send
    self.connect()
  File "/Users/cla/opt/anaconda3/lib/python3.7/http/client.py", lin

In [379]:
destination_df

,Starting_Point,First_Stop,Second_Stop,Third_Stop,End_Point,location,point,location2,point2
1468,miami,paris,tel aviv,madrid,miami,"(Miami, Miami-Dade County, Florida, United Sta...","(25.7742658, -80.1936589, 0.0)",None,None
1901,miami,tel aviv,paris,madrid,miami,"(Miami, Miami-Dade County, Florida, United Sta...","(25.7742658, -80.1936589, 0.0)","(תל אביב-יפו, מחוז תל אביב, ישראל, (32.0804808...","(32.0804808, 34.7805274, 0.0)"
4220,miami,madrid,tel aviv,paris,miami,"(Miami, Miami-Dade County, Florida, United Sta...","(25.7742658, -80.1936589, 0.0)","(Madrid, Área metropolitana de Madrid y Corred...","(40.4167047, -3.7035825, 0.0)"
251,miami,tel aviv,madrid,paris,miami,"(Miami, Miami-Dade County, Florida, United Sta...","(25.7742658, -80.1936589, 0.0)","(תל אביב-יפו, מחוז תל אביב, ישראל, (32.0804808...","(32.0804808, 34.7805274, 0.0)"
63,miami,paris,madrid,tel aviv,miami,"(Miami, Miami-Dade County, Florida, United Sta...","(25.7742658, -80.1936589, 0.0)","(Paris, Île-de-France, France, (48.8566101, 2....","(48.8566101, 2.3514992, 0.0)"
451,miami,madrid,paris,tel aviv,miami,"(Miami, Miami-Dade County, Florida, United Sta...","(25.7742658, -80.1936589, 0.0)","(Madrid, Área metropolitana de Madrid y Corred...","(40.4167047, -3.7035825, 0.0)"


In [402]:
for citi in cities:
    print(citi)
    location = geolocator.geocode(citi)

miami


GeocoderTimedOut: Service timed out